<a href="https://colab.research.google.com/github/BYU-Hydroinformatics/gwbf-notebooks/blob/main/3_GLSB_filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1.Sub basin Buffer

In [16]:
# subbasin
import geopandas as gpd

subbasin_gdf = gpd.read_file('/content/GSLB_subbasin.shp')

print(subbasin_gdf.head())
type(subbasin_gdf)
print(subbasin_gdf.crs)

     HYBAS_ID   NEXT_DOWN   NEXT_SINK    MAIN_BAS  DIST_SINK  DIST_MAIN  \
0  7120591510  7120593250  7120008710  7120008710      969.2      969.2   
1  7120591660  7120593250  7120008710  7120008710      969.3      969.3   
2  7121092250  7121093050  7120008710  7120008710      990.5      990.5   
3  7121091800  7121092250  7120008710  7120008710      997.0      997.0   
4  7120585470  7121091800  7120008710  7120008710     1003.6     1003.6   

   SUB_AREA  UP_AREA       PFAF_ID  ENDO  COAST  ORDER  SORT  \
0     124.3    909.1  772344093100     0      0      3  8593   
1     146.5    146.7  772344092000     0      0      4  8594   
2     155.7    689.3  772344093300     0      0      3  8643   
3     152.7    533.6  772344093400     0      0      3  8650   
4     131.7    269.3  772344095100     0      0      3  8663   

                                            geometry  
0  MULTIPOLYGON (((-114.05437 37.96741, -114.0546...  
1  POLYGON ((-114.03506 37.95776, -114.0353 37.95...  

In [17]:
# gage
import pandas as pd

gage_df = pd.read_csv('/content/gsl_nwm_gage.csv')
print(gage_df.head())
type(gage_df)

         id                                               name  \
0  10011200  WEST FORK BEAR RIVER AT WHITNEY DAM - NR OAKLE...   
1  10011500            BEAR RIVER NEAR UTAH-WYOMING STATE LINE   
2  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...   
3  10015900       SULPHUR CREEK BL RES. - NEAR EVANSTON - WYO.   
4  10016900                        BEAR RIVER AT EVANSTON - WY   

                  River   latitude   longitude  elevation_m    state  
0  WEST FORK BEAR RIVER  40.841614 -110.927119         2797     Utah  
1            BEAR RIVER  40.965225 -110.853508         2432     Utah  
2         SULPHUR CREEK  41.129114 -110.806563         2205  Wyoming  
3         SULPHUR CREEK  41.156058 -110.835176         2173  Wyoming  
4            BEAR RIVER  41.270224 -110.963793         2057  Wyoming  


pandas.core.frame.DataFrame

In [18]:
# well
well_gdf = gpd.read_file('/content/well_shp.shp')
print(well_gdf.head())
type(well_gdf)
print(well_gdf.crs)

        Well_ID          Well_Name    lat_dec    long_dec     GSE  AquiferID  \
0  3.810331e+14  (C-30-18)25aad- 1  38.175796 -113.802750  7098.0          1   
1  3.810371e+14  (C-30-17)30bab- 1  38.176306 -113.795500  7193.0          1   
2  3.811521e+14  (C-30-17)15cab- 1  38.197833 -113.741167  6550.0          1   
3  3.812361e+14  (C-30-18)12cdb- 1  38.210028 -113.815500  7190.0          1   
4  3.821131e+14  (C-28-17)22dda- 1  38.353571 -113.732473  5775.0          1   

  Aquifer_Na State                     geometry  
0  GSL Basin    UT   POINT (-113.80275 38.1758)  
1  GSL Basin    UT   POINT (-113.7955 38.17631)  
2  GSL Basin    UT  POINT (-113.74117 38.19783)  
3  GSL Basin    UT   POINT (-113.8155 38.21003)  
4  GSL Basin    UT  POINT (-113.73247 38.35357)  
EPSG:4326


EPSG:4326


In [36]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from typing import Dict, List, Tuple

def batch_process_gage_well_relationships(
    subbasins_shp: str,
    gage_csv: str,
    wells_csv: str
) -> Tuple[pd.DataFrame, Dict]:
    """
    Process all gages and find their corresponding wells in subbasins

    Returns:
    --------
    Tuple containing:
    - DataFrame with one row per gage-well relationship
    - Dictionary with summary statistics
    """
    # Read all data
    print("Reading input files...")
    subbasins = gpd.read_file(subbasins_shp)
    gages = pd.read_csv(gage_csv)
    wells = pd.read_csv(wells_csv)

    # Create GeoDataFrames
    gages_gdf = gpd.GeoDataFrame(
        gages,
        geometry=[Point(xy) for xy in zip(gages['longitude'], gages['latitude'])],
        crs="EPSG:4326"
    )

    wells_gdf = gpd.GeoDataFrame(
        wells,
        geometry=[Point(xy) for xy in zip(wells['long_dec'], wells['lat_dec'])],
        crs="EPSG:4326"
    )

    # Initialize results
    relationships_data = []
    summary_stats = {
        'total_gages': len(gages),
        'total_wells': len(wells),
        'gages_with_wells': 0,
        'total_relationships': 0,
        'wells_matched': set()
    }

    # Process each gage
    print("Processing gages...")
    for idx, gage in gages_gdf.iterrows():
        # Find containing subbasin for this gage
        containing_subbasin = None
        for _, subbasin in subbasins.iterrows():
            if subbasin.geometry.contains(gage.geometry):
                containing_subbasin = subbasin
                break

        if containing_subbasin is None:
            continue

        # Find wells in this subbasin
        wells_found = False
        for _, well in wells_gdf.iterrows():
            if containing_subbasin.geometry.contains(well.geometry):
                wells_found = True
                summary_stats['wells_matched'].add(well['Well_ID'])

                # Add relationship to results
                relationships_data.append({
                    # Gage information
                    'gage_id': gage['id'],
                    'gage_name': gage['name'],
                    'gage_latitude': gage['latitude'],
                    'gage_longitude': gage['longitude'],
                    'gage_elevation_m': gage['elevation_m'],
                    'river': gage['River'],
                    'gage_state': gage['state'],

                    # Subbasin information
                    'hybas_id': containing_subbasin['HYBAS_ID'],
                    'next_down': containing_subbasin['NEXT_DOWN'],
                    'sub_area': containing_subbasin['SUB_AREA'],
                    'up_area': containing_subbasin['UP_AREA'],
                    'stream_order': containing_subbasin['ORDER'],

                    # Well information
                    'well_id': well['Well_ID'],
                    'well_name': well['Well_Name'],
                    'well_latitude': well['lat_dec'],
                    'well_longitude': well['long_dec'],
                    'well_gse': well['GSE'],
                    'aquifer_id': well['AquiferID'],
                    'aquifer_name': well['Aquifer_Name'],
                    'well_state': well['State']
                })

        if wells_found:
            summary_stats['gages_with_wells'] += 1

    # Create DataFrame from relationships
    relationships_df = pd.DataFrame(relationships_data)

    # Update summary statistics
    summary_stats['total_relationships'] = len(relationships_df)
    summary_stats['wells_matched'] = len(summary_stats['wells_matched'])

    # Print summary
    print("\nProcessing Summary:")
    print(f"Total gages processed: {summary_stats['total_gages']}")
    print(f"Gages with matching wells: {summary_stats['gages_with_wells']}")
    print(f"Total wells matched: {summary_stats['wells_matched']}")
    print(f"Total relationships found: {summary_stats['total_relationships']}")

    return relationships_df, summary_stats

if __name__ == "__main__":
    # Process all relationships
    relationships_df, stats = batch_process_gage_well_relationships(
        'GSLB_subbasin.shp',
        'gsl_nwm_gage.csv',
        'GSLB_1900-2023_wells_with_aquifers.csv'
    )

    # Display first few relationships
    print("\nFirst few gage-well relationships:")
    print(relationships_df.head())

    # Save results
    relationships_df.to_csv('gage_well_relationships.csv', index=False)
    print("\nResults saved to 'gage_well_relationships.csv'")

Reading input files...
Processing gages...

Processing Summary:
Total gages processed: 78
Gages with matching wells: 69
Total wells matched: 2218
Total relationships found: 2663

First few gage-well relationships:
    gage_id                                          gage_name  gage_latitude  \
0  10011500            BEAR RIVER NEAR UTAH-WYOMING STATE LINE      40.965225   
1  10011500            BEAR RIVER NEAR UTAH-WYOMING STATE LINE      40.965225   
2  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...      41.129114   
3  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...      41.129114   
4  10015700  SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...      41.129114   

   gage_longitude  gage_elevation_m          river gage_state    hybas_id  \
0     -110.853508              2432     BEAR RIVER       Utah  7120514120   
1     -110.853508              2432     BEAR RIVER       Utah  7120514120   
2     -110.806563              2205  SULPHUR CREEK    Wyoming  71205

In [37]:
relationships_df.head()

,gage_id,gage_name,gage_latitude,gage_longitude,gage_elevation_m,river,gage_state,hybas_id,next_down,sub_area,up_area,stream_order,well_id,well_name,well_latitude,well_longitude,well_gse,aquifer_id,aquifer_name,well_state
0,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,40.965225,-110.853508,2432,BEAR RIVER,Utah,7120514120,7120511450,60.6,513.4,1,410154110533201,12-120-12daa01,41.031614,-110.892954,7580.0,1,GSL Basin,WY
1,10011500,BEAR RIVER NEAR UTAH-WYOMING STATE LINE,40.965225,-110.853508,2432,BEAR RIVER,Utah,7120514120,7120511450,60.6,513.4,1,410218110541501,12-120-12ba 01,41.038281,-110.904899,7538.0,1,GSL Basin,WY
2,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,41.129114,-110.806563,2205,SULPHUR CREEK,Wyoming,7120511280,7121059030,207.7,207.8,2,410341110495002,13-119-34cbb02,41.061336,-110.831286,7533.0,1,GSL Basin,WY
3,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,41.129114,-110.806563,2205,SULPHUR CREEK,Wyoming,7120511280,7121059030,207.7,207.8,2,410531110502401,13-119-21ac 01,41.091892,-110.840731,7428.0,1,GSL Basin,WY
4,10015700,SULPHUR CR.AB.RES.BL.LA CHAPELLE CR.NR EVANSTO...,41.129114,-110.806563,2205,SULPHUR CREEK,Wyoming,7120511280,7121059030,207.7,207.8,2,410611110473201,13-119-13cb 01,41.103003,-110.792952,7538.0,1,GSL Basin,WY


# 2.Elevation buffer

In [39]:
final_measurements_delta = pd.read_csv('final_measurements_delta.csv')
final_measurements_delta.head()

,Well_ID,Date,WTE,Delta_WTE
0,381033113480701,2012-09-06,7092.990,0.000
1,381037113474001,2012-09-06,7175.950,0.000
2,381152113442801,1995-11-22,6200.000,0.000
3,381236113485601,2014-07-23,7151.000,0.000
4,382113113435401,2018-05-25,5397.996,2.046


In [41]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time

def apply_elevation_buffer(
    relationships_df: pd.DataFrame,
    measurements_df: pd.DataFrame,
    wte_buffer: float = 100  # buffer in meters
) -> pd.DataFrame:
    """
    Apply elevation buffer to gage-well relationships and combine with measurements
    """
    print("Starting processing...")
    time.sleep(0.5)  # Small delay to ensure progress bar displays properly

    # Progress bar 1: Data preparation
    with tqdm(total=3, desc="Preparing data") as pbar:
        # Select required columns from relationships
        base_cols = [
            'gage_id', 'gage_latitude', 'gage_longitude', 'gage_elevation_m',
            'hybas_id', 'next_down', 'well_id', 'well_name',
            'well_latitude', 'well_longitude', 'well_gse'
        ]
        base_df = relationships_df[base_cols].copy()
        pbar.update(1)

        # Merge with measurements
        print("\nMerging data...")
        merged = pd.merge(
            base_df,
            measurements_df,
            left_on='well_id',
            right_on='Well_ID',
            how='inner'
        )
        pbar.update(1)

        # Convert WTE to meters (if it's in feet)
        merged['WTE_meters'] = merged['WTE'] * 0.3048
        pbar.update(1)

    # Progress bar 2: Applying filters
    total_rows = len(merged)
    filtered_data = pd.DataFrame()

    print("\nApplying elevation buffer...")
    chunk_size = 10000  # Process in chunks to show progress
    chunks = [merged[i:i + chunk_size] for i in range(0, total_rows, chunk_size)]

    with tqdm(total=len(chunks), desc="Filtering data") as pbar:
        filtered_chunks = []
        for chunk in chunks:
            # Apply elevation buffer
            elevation_mask = chunk['WTE_meters'] > (chunk['gage_elevation_m'] - wte_buffer)
            filtered_chunk = chunk[elevation_mask]
            filtered_chunks.append(filtered_chunk)
            pbar.update(1)

        filtered_data = pd.concat(filtered_chunks, ignore_index=True)

    # Progress bar 3: Final processing
    with tqdm(total=2, desc="Finalizing output") as pbar:
        # Select and rename final columns
        final_cols = [
            'gage_id', 'gage_latitude', 'gage_longitude', 'gage_elevation_m',
            'hybas_id', 'next_down', 'well_id', 'well_name',
            'well_latitude', 'well_longitude', 'well_gse',
            'Date', 'WTE', 'Delta_WTE'
        ]

        result = filtered_data[final_cols].copy()
        pbar.update(1)

        # Rename columns to match requested format
        result = result.rename(columns={
            'Date': 'date',
            'WTE': 'wte',
            'Delta_WTE': 'delta_wte'
        })
        pbar.update(1)

    return result

def print_summary_stats(input_df: pd.DataFrame, output_df: pd.DataFrame):
    """Print summary statistics of the filtering process"""
    print("\nFiltering Summary:")
    print(f"Original relationships: {len(input_df)}")
    print(f"After elevation buffer: {len(output_df)}")
    print(f"Unique gages before: {input_df['gage_id'].nunique()}")
    print(f"Unique gages after: {output_df['gage_id'].nunique()}")
    print(f"Unique wells before: {input_df['well_id'].nunique()}")
    print(f"Unique wells after: {output_df['well_id'].nunique()}")

    # Additional statistics
    print("\nDetailed Statistics:")
    print(f"Number of measurements per well (average): "
          f"{len(output_df) / output_df['well_id'].nunique():.1f}")
    print(f"Number of wells per gage (average): "
          f"{output_df['well_id'].nunique() / output_df['gage_id'].nunique():.1f}")

if __name__ == "__main__":
    # Load data with progress bars
    print("Loading input files...")
    with tqdm(total=2, desc="Loading files") as pbar:
        relationships_df = pd.read_csv('gage_well_relationships.csv')
        pbar.update(1)
        measurements_df = pd.read_csv('final_measurements_delta.csv')
        pbar.update(1)

    # Apply elevation buffer
    result_df = apply_elevation_buffer(
        relationships_df,
        measurements_df,
        wte_buffer=100  # adjust buffer size as needed
    )

    # Print summary statistics
    print_summary_stats(relationships_df, result_df)

    # Display sample of results
    print("\nSample of filtered data:")
    print(result_df.head())

    # Save results
    print("\nSaving results...")
    with tqdm(total=1, desc="Saving file") as pbar:
        result_df.to_csv('filtered_gage_well_relationships.csv', index=False)
        pbar.update(1)
    print("Results saved to 'filtered_gage_well_relationships.csv'")

Loading input files...


Loading files:   0%|          | 0/2 [00:00<?, ?it/s]

Starting processing...


Preparing data:   0%|          | 0/3 [00:00<?, ?it/s]


Merging data...

Applying elevation buffer...


Filtering data:   0%|          | 0/592 [00:00<?, ?it/s]

Finalizing output:   0%|          | 0/2 [00:00<?, ?it/s]


Filtering Summary:
Original relationships: 2663
After elevation buffer: 3995214
Unique gages before: 69
Unique gages after: 34
Unique wells before: 2218
Unique wells after: 1178

Detailed Statistics:
Number of measurements per well (average): 3391.5
Number of wells per gage (average): 34.6

Sample of filtered data:
    gage_id  gage_latitude  gage_longitude  gage_elevation_m    hybas_id  \
0  10015700      41.129114     -110.806563              2205  7120511280   
1  10015700      41.129114     -110.806563              2205  7120511280   
2  10015700      41.129114     -110.806563              2205  7120511280   
3  10015700      41.129114     -110.806563              2205  7120511280   
4  10015700      41.129114     -110.806563              2205  7120511280   

    next_down          well_id       well_name  well_latitude  well_longitude  \
0  7121059030  410341110495002  13-119-34cbb02      41.061336     -110.831286   
1  7121059030  410531110502401  13-119-21ac 01      41.091892  

Saving file:   0%|          | 0/1 [00:00<?, ?it/s]

Results saved to 'filtered_gage_well_relationships.csv'
